In [1]:
from pprint import pprint

import sys
import os
import pandas as pd
# from dotenv import load_dotenv
import csv

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
import numpy as np

C:\Users\Jeffrey\AppData\Local\Temp\ipykernel_23768\1333719079.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
CURRENT_DIR = os.getcwd()

DATA_ROOT_DIR = os.path.abspath(os.path.join(CURRENT_DIR, "..", "dataset", "data_files"))
USER_GAMES_FILENAME = os.path.join(DATA_ROOT_DIR, "users_games.csv")

df = pd.read_csv(USER_GAMES_FILENAME)

pprint(df)

                 user_id  game_id  playtime_2weeks  playtime_forever
0      76561197962641822       10              NaN                 0
1      76561197962641822       20              NaN                 0
2      76561197962641822       30              NaN                 0
3      76561197962641822       40              NaN                 0
4      76561197962641822       50              NaN                 0
...                  ...      ...              ...               ...
33773  76561199615987653      730           3302.0              3985
33774  76561199622123452      730           1820.0              1820
33775  76561199625697401     3590            976.0               976
33776  76561199625697401  1966720            443.0               443
33777  76561199625697401  1721470             17.0                17

[33778 rows x 4 columns]


In [3]:
# Creates model from an array of phrases
def createDoc2VecModel(phrases):
    tagged_phrases = [TaggedDocument(words=simple_preprocess(phrase), tags=[i]) for i, phrase in enumerate(phrases)]

    model = Doc2Vec(vector_size=100, window=2, min_count=1, workers=4, epochs=20)
    model.build_vocab(tagged_phrases)
    model.train(tagged_phrases, total_examples=model.corpus_count, epochs=model.epochs)

    return model

# Function predicts from a given model and phrase
def predict_from_model(model, phrase):
    words = simple_preprocess(phrase)
    vector = model.infer_vector(words)

    return vector




In [4]:
SAVE_FILE_NAME = "test1"


# Training Data for now
phrases = [
    "This is true",
    "This is false",
    "Another phrase here",
    "Yet another phrase"
]

# Test data for now
p1 = "A new phrase"

train_model = createDoc2VecModel(phrases)
new_vector = predict_from_model(train_model, p1)

pprint(new_vector)

train_model.save(f"{SAVE_FILE_NAME}")


array([ 0.00485734, -0.00276227,  0.00498454,  0.00025802,  0.00332456,
       -0.00172008, -0.0021381 ,  0.00274478, -0.00174672,  0.00336556,
       -0.00438887,  0.00419236,  0.00482308, -0.00175918,  0.00133059,
        0.00474097, -0.0022495 ,  0.00147214,  0.00199189,  0.00015044,
        0.0033116 ,  0.00395688,  0.00381812,  0.00034643,  0.00067904,
        0.00108616,  0.00221621, -0.00384008, -0.00126196,  0.00259026,
        0.00176151,  0.00396385,  0.00065929,  0.00430007, -0.00439349,
        0.00493642, -0.00476743,  0.00494333, -0.00451056, -0.00335615,
        0.00035468, -0.00031224, -0.00268762,  0.0002627 , -0.0037834 ,
       -0.00415431,  0.00373667,  0.00452212, -0.00054107,  0.0046086 ,
       -0.00383992,  0.00401965, -0.00036801,  0.00261782,  0.00476794,
        0.00324471, -0.00162591, -0.00021468, -0.00403976,  0.00124021,
        0.00096194,  0.0025501 ,  0.00191706, -0.00063637,  0.00108182,
       -0.00289187,  0.00209754,  0.00312685, -0.00400469, -0.00

In [28]:
#### Testing with BERT

from transformers import BertTokenizer, BertModel
import torch

def generate_bert_embeddings(descriptions, model_name='bert-base-uncased'):
    """
    Generate BERT embeddings for a list of game descriptions.

    Args:
    descriptions (list): List of game descriptions (strings).
    model_name (str): Name of the pre-trained BERT model to use.

    Returns:
    embeddings (list): List of BERT embeddings for each description.
    """

    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    embeddings = []
    for description in descriptions:
        tokens = tokenizer.tokenize(description)
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_tensor = torch.tensor([input_ids])        # Pytorch tensor dtype

        with torch.no_grad():
            outputs = model(input_tensor)
            embeddings.append(outputs[0][:, 0, :].numpy())
            print(outputs[0][:, 0, :].numpy().shape, "\n")

    return embeddings





game_descriptions = [
    "At its core, Minecraft is a game where players place blocks and go on adventures. This includes anything from crafting simple items like containers or weapons, to building structures like houses, castles, and cities, or even making complex mechanical devices, all within the game's world.",
    "Terraria is a 2D sandbox game with gameplay that revolves around exploration, building, crafting, combat, survival, and mining, playable in both single-player and multiplayer modes. The game has a 2D sprite tile-based graphical style reminiscent of the 16-bit sprites found on the Super NES.",
    "At its core, Roblox is a gaming platform that allows you to play dozens of user-created games or create games through Roblox Studio. From role-playing games to rhythm titles and shooters, the platform is a toy box of experiences for all ages.",
    "League of Legends is a team-based strategy game where two teams of five powerful champions face off to destroy the other's base. Choose from over 140 champions to make epic plays, secure kills, and take down towers as you battle your way to victory.",
    '''
Minecraft is a game made up of blocks, creatures, and community. You can survive the night or build a work of art – the choice is all yours. But if the thought of exploring a vast new world all on your own feels overwhelming, then fear not! Let’s explore what Minecraft is all about!

WHAT IS THE GOAL OF MINECRAFT?
Minecraft has no set goal and can be played however you’d like! This is why it’s sometimes called a “sandbox game” – there are lots of things for you to do, and lots of ways that you can play. If you like being creative, then you can use the blocks to build things from your imagination. If you’re feeling brave, you can explore the world and face daring challenges. Blocks can be broken, crafted, placed to reshape the landscape, or used to build fantastical creations. 

Creatures can be battled or befriended, depending on how you play. The world of Minecraft allows for epic adventures, quiet meditations, and everything in between. You can even share your creations with other players, or play in community worlds!

Learn more about crafting, one of Minecraft’s basic features, in this guide. '''
]

embeddings = generate_bert_embeddings(game_descriptions)

# for i in range(len(game_descriptions)):
#     print(embeddings[i], '\n')


(1, 768) 

(1, 768) 

(1, 768) 

(1, 768) 

(1, 768) 

